In [52]:
# Calcula el pago de luz.

import pandas as pd
import numpy as np

In [53]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/"

# Escenarios.
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path + "data/" + x + "/data" for x in dirs]
csv = ".csv"
path_cfe = path + "CFE/"
f_tarifa = "Tarifa_1.csv"
f_dac = "DAC.csv"
f_regiones = "Regiones.csv"

tarifa = pd.read_csv(path_cfe + f_tarifa)
dac = pd.read_csv(path_cfe + f_dac)
regiones = pd.read_csv(path_cfe + f_regiones)

# Tipos de escalones para las tarifas.
escalon_0 = tarifa["Escalon"].unique()

# Se establece la tarifa DAC para cada estado.
regiones[ ["DAC", "Equilibrio"] ] = np.nan
for i in regiones.itertuples():
    regiones.loc[ i[0], ["DAC", "Equilibrio"] ] = ( 
        dac.drop(8).loc[ dac["Region"] == i.Region,
        ["DAC", "Equilibrio"] ].values[0] )


In [134]:
k = 0

cons = ["Verano_Consumo_1*", "Verano_Consumo_DAC",
    "Invierno_Consumo_1*", "Invierno_Consumo_DAC"]

# Información mensual.
df_0 = pd.read_csv(path_data[k] + csv)
# Información anual.
df_y = pd.read_csv(path_data[k] + "_yearly" + csv)

# Promediamos el consumo y usuarios anuales.
if k == 0:
    df = df_y.groupby("CVE_INEGI").mean()
    df["Tarifa"] = ""

    # Recuperamos la información de las tarifas.
    for i in df.itertuples():
        df.loc[i[0], "Tarifa"] = df_y.query(
            "CVE_INEGI==" + str(i[0]) +
            " & Año==2010" )["Tarifa"].values[0]

# Redondeamos la cantidad de usuarios.
df[["Usuarios_1*", "Usuarios_DAC"]] = df[
    ["Usuarios_1*", "Usuarios_DAC"]].round()

# Retiramos la columna de año, que ya no tiene sentido.
df.drop(columns = "Año", inplace = True)

# Sumamos el consumo correspondiente al verano.
df_2 = df_0.where(df_0["Verano"] == True)
df_2 = df_2[df_2["CVE_INEGI"].notna()]
df_2 = df_2.groupby(["CVE_INEGI", "Mes"]
    ).mean().reset_index()
df_2 = df_2.groupby(["CVE_INEGI"]
    ).sum()#.reset_index()
df[cons[0:2]] = df_2[["Consumo_1*", "Consumo_DAC"]]

# Sumamos el consumo correspondiente al invierno.
df_2 = df_0.where(df_0["Verano"] == False)
df_2 = df_2[df_2["CVE_INEGI"].notna()]
df_2 = df_2.groupby(["CVE_INEGI", "Mes"]
    ).mean().reset_index()
df_2 = df_2.groupby(["CVE_INEGI"]
    ).sum()#.reset_index()
df[cons[2:4]] = df_2[["Consumo_1*", "Consumo_DAC"]]

# Retiramos las columnas de consumo total anual.
df.drop(columns = ["Consumo_1*", "Consumo_DAC"], inplace = True)

# Unimos las tarifas 1 y DAC.
df["Verano_Consumo_1*"] += df["Verano_Consumo_DAC"]
df.drop("Verano_Consumo_DAC", axis = 1, inplace = True)
df["Invierno_Consumo_1*"] += df["Invierno_Consumo_DAC"]
df.drop("Invierno_Consumo_DAC", axis = 1, inplace = True)
df["Usuarios_1*"] += df["Usuarios_DAC"]
df.drop("Usuarios_DAC", axis = 1, inplace = True)

df.reset_index(inplace = True)

df["Verano_Consumo_1*"] += 4e8

# Se crean columnas para los
# escalones de verano e invierno.
verano_escalon = []
invierno_escalon = []
for e in escalon_0:
    verano_escalon.append("Verano_" + e)
    invierno_escalon.append("Invierno_" + e)
escalon = [verano_escalon,
    invierno_escalon]
df[verano_escalon] = np.nan
df[invierno_escalon] = np.nan

# Se establece el valor de consumo límite para cada escalon.
for i in df.itertuples():
    for p in range(len(escalon_0)):
        df.loc[i[0], verano_escalon[p]] = ( 
            tarifa.query('Temporada=="Verano"'
            ' & Escalon=="' + escalon_0[p] + '" & Tarifa=="'
            + str(i.Tarifa) + '"').iat[0, 4] * 6 )
        df.loc[i[0], invierno_escalon[p]] = ( 
            tarifa.query('Temporada=="Verano"'
            ' & Escalon=="' +  escalon_0[p] + '" & Tarifa=="'
            + str(i.Tarifa) + '"').iat[0, 4] * 6 )

# Columnas para consumo en verano e invierno.
estacion = ["Verano_", "Invierno_"]
verano_escalon_cons = [ "Cons_" +
    x for x in verano_escalon ]
invierno_escalon_cons = [ "Cons_" +
    x for x in invierno_escalon ]
escalon_cons = [ verano_escalon_cons,
    invierno_escalon_cons ]

# Columnas para verano.
df["Verano_Consumo_Usuario"] = ( df["Verano_Consumo_1*"]
    / df["Usuarios_1*"] )
df[escalon_cons[0]] = df[escalon[0]].copy()
#df[estacion[0] + "Cons_DAC"] = np.nan
df[estacion[0] + "Pago"] = 0

# Columnas para invierno.
df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
    / df["Usuarios_1*"] )
df[escalon_cons[1]] = df[escalon[1]].copy()
#df[estacion[1] + "Cons_DAC"] = np.nan
df[estacion[1] + "Pago"] = 0

df["DAC_Pago"] = 0

df

,CVE_INEGI,CVE_ENT,CVE_MUN,Usuarios_1*,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Tarifa,Verano_Consumo_1*,Invierno_Consumo_1*,Verano_Basico,Verano_Intermedio_bajo,Verano_Intermedio_alto,Verano_Excedente,Invierno_Basico,Invierno_Intermedio_bajo,Invierno_Intermedio_alto,Invierno_Excedente,Verano_Consumo_Usuario,Cons_Verano_Basico,Cons_Verano_Intermedio_bajo,Cons_Verano_Intermedio_alto,Cons_Verano_Excedente,Verano_Pago,Invierno_Consumo_Usuario,Cons_Invierno_Basico,Cons_Invierno_Intermedio_bajo,Cons_Invierno_Intermedio_alto,Cons_Invierno_Excedente,Invierno_Pago,DAC_Pago
0,1001,1.0,1.0,254974.0,-102.295872,21.811400,27.182119,13.074257,20.159075,522.051880,-482.033417,507.881513,-8.212147,1.347620,442.348606,845118.000000,1.374286e+11,23.691382,4.142857,7.245778,162160.723559,NaN,11.211926,116635.924759,1,5.920337e+08,1.693105e+08,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,2321.937399,450.0,390.0,0.0,1500.0,0,664.030341,450.0,390.0,0.0,1500.0,0,0
1,1002,1.0,2.0,11557.0,-102.045590,22.126479,26.460970,12.328507,19.395846,429.399714,-494.662554,556.378653,-12.173154,0.947158,358.376980,46075.142857,3.491166e+09,23.034981,4.142857,0.847945,75705.155679,NaN,11.211926,54337.426603,1,4.077595e+08,6.846582e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,35282.471832,450.0,390.0,0.0,1500.0,0,592.418585,450.0,390.0,0.0,1500.0,0,0
2,1003,1.0,3.0,16736.0,-102.704911,21.900640,27.009948,12.518081,19.773620,621.113330,-448.972848,462.771497,-4.712476,0.947242,519.512183,55283.142857,4.358511e+09,23.082358,4.142857,0.598719,78733.145869,NaN,11.211926,92335.783229,1,4.112233e+08,9.884636e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,24571.180486,450.0,390.0,0.0,1500.0,0,590.621186,450.0,390.0,0.0,1500.0,0,0
3,1004,1.0,4.0,3788.0,-102.297038,22.360619,26.527926,12.449918,19.528949,448.636825,-512.037785,557.611886,-13.815264,0.775050,377.125221,15363.000000,1.449504e+09,23.041425,4.142857,1.196649,94221.940180,NaN,11.211926,12838.354454,1,4.025108e+08,2.213835e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,106259.444671,450.0,390.0,0.0,1500.0,0,584.433834,450.0,390.0,0.0,1500.0,0,0
4,1005,1.0,5.0,30597.0,-102.445700,21.932112,26.925495,12.886637,19.890029,531.063800,-460.285867,479.792441,-7.156598,1.232661,442.897088,112079.000000,1.318374e+10,23.204577,4.142857,2.242050,117305.832917,NaN,11.211926,49989.509158,1,4.214123e+08,1.878628e+07,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,13772.994903,450.0,390.0,0.0,1500.0,0,613.990777,450.0,390.0,0.0,1500.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32.0,54.0,3923.0,-101.714324,22.407345,26.185634,12.639281,19.411589,357.411519,-506.235879,590.748675,-22.991064,1.840217,288.391072,18889.000000,1.104084e+09,23.055471,4.000000,0.508554,58432.484498,NaN,11.268627,37142.598662,1,4.020974e+08,1.593920e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,102497.422124,450.0,390.0,0.0,1500.0,0,406.301266,450.0,390.0,0.0,1500.0,0,0
2465,32055,32.0,55.0,8847.0,-102.851571,22.298130,26.407706,12.344591,19.528930,558.532327,-469.145342,489.120322,-9.259408,0.741970,468.364202,29902.000000,2.396876e+09,22.681947,4.142857,0.138660,80134.667679,NaN,11.268627,215650.560221,1,4.051272e+08,3.901293e+06,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,45792.609705,450.0,390.0,0.0,1500.0,0,440.973591,450.0,390.0,0.0,1500.0,0,0
2466,32056,32.0,56.0,37702.0,-102.678055,22.735730,25.725354,11.868202,18.824115,439.723002,-530.293745,536.387068,-19.077788,1.257958,374.477437,142958.571429,2.484715e+10,22.429208,4.142857,3.272704,173750.774044,NaN,11.268627,43682.101050,1,4.261100e+08,1.966034e+07,450.0,390.0,0.0,1500.0,450.0,390.0,0.0,1500.0,11302.052643,450.0,390.0,0.0,1500.0,0,521.466643,450.0,390.0,0.0,1500.0,0,0
2467,32057,32.0,57.0,15674.0,-102.3

In [135]:
df_2 = df.copy()

In [136]:
df = df_2.copy()

IVA = 1.16
FIJO_DAC = 121.75

for e in [0, 1]:

    if e == 0: s = 0
    else: s = 6

    col = np.concatenate( [
        [estacion[e] + "Consumo_Usuario"],
        escalon_cons[e]
        ] )

    for i in range(len(col[:-1])):
        df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

    df[col[1:]] = df[col[1:]].clip( lower = 0 )

    col = np.concatenate( [ escalon_cons[e] ] )
    col = np.flip(col)

    for i in range(len(col[:-1])):
        df[col[i]] = df[col[i + 1]]

    for i in df.itertuples():
            
        if not( ( i.Verano_Consumo_Usuario +
            i.Invierno_Consumo_Usuario ) > i.Verano_Excedente):
            if i[37 + s] == 0:
                df.loc[i[0], escalon_cons[e][0]] = i[36 + s]
            if i.Tarifa in ["1", "1A", "1B"]:
                df.loc[i[0], escalon_cons[e][2]] = 0
            for j in range(3):
                if i[37 + s + j + 1] > 0:
                    df.loc[i[0], escalon_cons[e][j]] = i[
                        28 + s * 4 // 6 + j]
        else:
            df.loc[i[0], col] = 0

            df.loc[i[0], "DAC_Pago"] = ( (
                i.Verano_Consumo_Usuario +
                i.Invierno_Consumo_Usuario )
                * regiones.loc[
                regiones["CVE_ENT"] == i.CVE_ENT,
                "DAC"].values[0] + FIJO_DAC )

    df[estacion[e] + "Pago"] = ( df[
        np.concatenate( [ escalon_cons[e]
        ] ) ].sum(axis = 1) * IVA )

df.drop( columns = np.concatenate( [
    escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
    ["Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
    ] ), inplace = True )

df["Pago_real"] = ( df[[estacion[0] + "Pago",
    estacion[1] + "Pago", "DAC_Pago"]].sum(axis = 1) )

df.drop( columns = [ estacion[0] + "Pago",
    estacion[1] + "Pago", "DAC_Pago" ], inplace = True )

# Unimos el consumo de verano e invierno.
df["Consumo_1*"] = df["Verano_Consumo_1*"] + df["Invierno_Consumo_1*"]
df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
df["Pago_sin_subsidio"] = np.nan

for i in df.itertuples():
    df.loc[i[0], "Pago_sin_subsidio"] = ( 
        i.Consumo_Usuario * regiones.loc[
        regiones["CVE_ENT"] == i.CVE_ENT,
        "Equilibrio" ].values[0] * IVA )

df["Subsidio_Usuario"] = ( df["Pago_sin_subsidio"] -
    df["Pago_real"] )
df["Subsidio"] = df["Subsidio_Usuario"] * df["Usuarios_1*"]

df.to_csv(path_data[k] + "_Pago" + csv)

df

,CVE_INEGI,CVE_ENT,CVE_MUN,Usuarios_1*,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Tarifa,Verano_Consumo_1*,Invierno_Consumo_1*,Pago_real,Consumo_1*,Consumo_Usuario,Pago_sin_subsidio,Subsidio_Usuario,Subsidio
0,1001,1.0,1.0,254974.0,-102.295872,21.811400,27.182119,13.074257,20.159075,522.051880,-482.033417,507.881513,-8.212147,1.347620,442.348606,845118.000000,1.374286e+11,23.691382,4.142857,7.245778,162160.723559,NaN,11.211926,116635.924759,1,5.920337e+08,1.693105e+08,1.558608e+04,7.613441e+08,2985.967740,1.037948e+04,-5206.593570,-1.327546e+09
1,1002,1.0,2.0,11557.0,-102.045590,22.126479,26.460970,12.328507,19.395846,429.399714,-494.662554,556.378653,-12.173154,0.947158,358.376980,46075.142857,3.491166e+09,23.034981,4.142857,0.847945,75705.155679,NaN,11.211926,54337.426603,1,4.077595e+08,6.846582e+06,1.859178e+05,4.146061e+08,35874.890417,1.247042e+05,-61213.570724,-7.074452e+08
2,1003,1.0,3.0,16736.0,-102.704911,21.900640,27.009948,12.518081,19.773620,621.113330,-448.972848,462.771497,-4.712476,0.947242,519.512183,55283.142857,4.358511e+09,23.082358,4.142857,0.598719,78733.145869,NaN,11.211926,92335.783229,1,4.112233e+08,9.884636e+06,1.304347e+05,4.211079e+08,25161.801673,8.746461e+04,-42970.111598,-7.191478e+08
3,1004,1.0,4.0,3788.0,-102.297038,22.360619,26.527926,12.449918,19.528949,448.636825,-512.037785,557.611886,-13.815264,0.775050,377.125221,15363.000000,1.449504e+09,23.041425,4.142857,1.196649,94221.940180,NaN,11.211926,12838.354454,1,4.025108e+08,2.213835e+06,5.534662e+05,4.047246e+08,106843.878505,3.713986e+05,-182067.589981,-6.896720e+08
4,1005,1.0,5.0,30597.0,-102.445700,21.932112,26.925495,12.886637,19.890029,531.063800,-460.285867,479.792441,-7.156598,1.232661,442.897088,112079.000000,1.318374e+10,23.204577,4.142857,2.242050,117305.832917,NaN,11.211926,49989.509158,1,4.214123e+08,1.878628e+07,7.463195e+04,4.401986e+08,14386.985680,5.001041e+04,-24621.536333,-7.533451e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32.0,54.0,3923.0,-101.714324,22.407345,26.185634,12.639281,19.411589,357.411519,-506.235879,590.748675,-22.991064,1.840217,288.391072,18889.000000,1.104084e+09,23.055471,4.000000,0.508554,58432.484498,NaN,11.268627,37142.598662,1,4.020974e+08,1.593920e+06,5.249307e+05,4.036913e+08,102903.723390,3.522459e+05,-172684.812540,-6.774425e+08
2465,32055,32.0,55.0,8847.0,-102.851571,22.298130,26.407706,12.344591,19.528930,558.532327,-469.145342,489.120322,-9.259408,0.741970,468.364202,29902.000000,2.396876e+09,22.681947,4.142857,0.138660,80134.667679,NaN,11.268627,215650.560221,1,4.051272e+08,3.901293e+06,2.359130e+05,4.090285e+08,46233.583295,1.582605e+05,-77652.557077,-6.869922e+08
2466,32056,32.0,56.0,37702.0,-102.678055,22.735730,25.725354,11.868202,18.824115,439.723002,-530.293745,536.387068,-19.077788,1.257958,374.477437,142958.571429,2.484715e+10,22.429208,4.142857,3.272704,173750.774044,NaN,11.268627,43682.101050,1,4.261100e+08,1.966034e+07,6.042170e+04,4.457703e+08,11823.519286,4.047265e+04,-19949.047116,-7.521190e+08
2467,32057,32.0,57.0,15674.0,-102.312009,22.755625,26.292193,12.041775,19.151676,404.788083,-550.953892,588.182495,-21.708939,0.905130,344.888589,18421.428571,1.414295e+09,22.864307,4.000000,0.843291,76845.878314,NaN,11.268627,21844.700672,1,4.114349e+08,9.139649e+06,1.369681e+05,4.205746e+08,26832.624665,9.184976e+04,-45118.371460,-7.071854e+08


In [130]:
f"{df['Subsidio'].sum():e}"

'1.162155e+11'

In [ ]:
k = 0

df_0 = pd.read_csv(path_data[k] + csv)

if k == 0:
    df = df_0.groupby("CVE_INEGI").mean()
    df.reset_index(inplace = True)
    df["Tarifa"] = ""

    for i in df.itertuples():
        df.loc[i[0], "Tarifa"] = df_0.query(
            "CVE_INEGI==" + str(i.CVE_INEGI) +
            " & Año==2010" )["Tarifa"].values[0]

df.drop(columns = "Año", inplace = True)

# Unimos las tarifas 1 y DAC.
df["Verano_Consumo_1*"] += df["Verano_Consumo_DAC"]
df.drop("Verano_Consumo_DAC", axis = 1, inplace = True)

df["Invierno_Consumo_1*"] += df["Invierno_Consumo_DAC"]
df.drop("Invierno_Consumo_DAC", axis = 1, inplace = True)

df["Usuarios_1*"] += df["Usuarios_DAC"]
df.drop("Usuarios_DAC", axis = 1, inplace = True)

verano_escalon = []
invierno_escalon = []
escalon = [verano_escalon,
    invierno_escalon]

for e in escalon_0:
    verano_escalon.append("Verano_" + e)
    invierno_escalon.append("Invierno_" + e)

df[verano_escalon] = np.nan
df[invierno_escalon] = np.nan

for i in df.itertuples():
    for p in range(len(escalon_0)):
        df.loc[i[0], verano_escalon[p]] = ( 
            tarifa.query('Temporada=="Verano"'
            ' & Escalon=="' + escalon_0[p] + '" & Tarifa=="'
            + str(i.Tarifa) + '"').iat[0, 4] * 12 )
        df.loc[i[0], invierno_escalon[p]] = ( 
            tarifa.query('Temporada=="Verano"'
            ' & Escalon=="' +  escalon_0[p] + '" & Tarifa=="'
            + str(i.Tarifa) + '"').iat[0, 4] * 12 )

estacion = ["Verano_", "Invierno_"]
verano_escalon_cons = [ "Cons_" +
    x for x in verano_escalon ]
invierno_escalon_cons = [ "Cons_" +
    x for x in invierno_escalon ]
escalon_cons = [ verano_escalon_cons,
    invierno_escalon_cons ]

df["Verano_Consumo_Usuario"] = ( df["Verano_Consumo_1*"]
    / df["Usuarios_1*"] )
df[escalon_cons[0]] = df[escalon[0]].copy()
#df[estacion[0] + "DAC"] = np.nan
df[estacion[0] + "Cons_DAC"] = np.nan
df[estacion[0] + "Pago"] = np.nan

df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
    / df["Usuarios_1*"] )
df[escalon_cons[1]] = df[escalon[1]].copy()
#df[estacion[1] + "DAC"] = np.nan
df[estacion[1] + "Cons_DAC"] = np.nan
df[estacion[1] + "Pago"] = np.nan

df.head()

KeyboardInterrupt: ignored

In [ ]:
IVA = 1.16
FIJO_DAC = 121.75

for e in [0, 1]:

    if e == 0: s = 0
    else: s = 7

    col = np.concatenate( [
        [estacion[e] + "Consumo_Usuario"],
        escalon_cons[e]
        ] )

    for i in range(len(col[:-1])):
        df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

    df[col[1:]] = df[col[1:]].clip( lower = 0 )

    col = np.concatenate( [ escalon_cons[e],
        [estacion[e] + "Cons_DAC"] ] )
    col = np.flip(col)

    for i in range(len(col[:-1])):
        df[col[i]] = df[col[i + 1]]

    for i in df.itertuples():
        if i[35 + s] == 0:
            df.loc[i[0], escalon_cons[e][0]] = i[34 + s]
        if i.Tarifa in ["1", "1A", "1B"]:
            df.loc[i[0], escalon_cons[e][2]] = 0
        for j in range(4):
            if i[35 + s + j + 1] > 0:
                df.loc[i[0], escalon_cons[e][j]] = i[35 + s + j]

    for i in df.itertuples():
        if e == 0:
            m = i.Verano_Cons_DAC
            n = i.Verano_Consumo_Usuario
            temp = "Verano"
        else:
            m = i.Invierno_Cons_DAC
            n = i.Invierno_Consumo_Usuario
            temp = "Invierno"
        if m == 0:
            for j in range(4):
                df.loc[i[0], escalon_cons[e][j]] = (
                    i[35 + s + j] * tarifa.query( 'Temporada=="' + temp
                    + '" & Escalon=="' + escalon_0[j] + '" & Tarifa=="'
                    + str(i.Tarifa) + '"' ).iat[0, 3] )
        else:
            df.loc[i[0], escalon_cons[e]] = 0
            df.loc[i[0], estacion[e] + "Cons_DAC"] = (
                n * regiones.loc[
                regiones["CVE_ENT"] == i.CVE_ENT,
                "DAC"].values[0] + FIJO_DAC )

    df[estacion[e] + "Pago"] = ( df[
        np.concatenate( [ escalon_cons[e],
        [estacion[e] + "Cons_DAC"] ] ) ].sum(axis = 1) * IVA )

df.drop( columns = np.concatenate( [
    escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
    [estacion[0] + "Cons_DAC", estacion[1] + "Cons_DAC",
    "Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
    ] ), inplace = True )

df["Pago_real"] = ( df[estacion[0] + "Pago"] 
    + df[estacion[1] + "Pago"] )

df.drop( columns = [ estacion[0] + "Pago",
    estacion[1] + "Pago" ], inplace = True )

# Unimos el consumo de verano e invierno.
df["Consumo_1*"] = df["Verano_Consumo_1*"] + df["Invierno_Consumo_1*"]
df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
df["Pago_sin_subsidio"] = np.nan

for i in df.itertuples():
    df.loc[i[0], "Pago_sin_subsidio"] = ( 
        i.Consumo_Usuario * regiones.loc[
        regiones["CVE_ENT"] == i.CVE_ENT,
        "Equilibrio" ].values[0] * IVA )

df.drop( columns = ["Consumo_1*"],
    inplace = True )

df["Subsidio_Usuario"] = ( df["Pago_sin_subsidio"] -
    df["Pago_real"] )
df["Subsidio"] = df["Subsidio_Usuario"] * df["Usuarios_1*"]

df.to_csv(path_data[k] + "_Pago" + csv)

df

,CVE_INEGI,CVE_ENT,CVE_MUN,Verano_Consumo_1*,Invierno_Consumo_1*,Usuarios_1*,lon,lat,Tmax,Tmin,Tmean,Pre,HDD_mean,CDD_mean,HDD_p10,CDD_p90,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$luz,$GLP,Area,Tarifa,Pago_real,Consumo_Usuario,Pago_sin_subsidio,Subsidio_Usuario,Subsidio_Per,Subsidio
0,1001,1.0,1.0,2.063279e+08,1.031640e+08,254973.000000,-102.295872,27.204040,13.124972,20.164510,-448.638681,560.865487,-448.638681,560.865487,521.520448,461.817592,NaN,845118.000000,1.374286e+11,7.245778,162160.723571,NaN,11.211926,116635.92480,1,1226.397279,1213.822082,4219.351042,2992.953764,0.709340,7.631224e+08
1,1002,1.0,2.0,8.287204e+06,4.143602e+06,11557.285714,-102.045590,26.476119,12.328314,19.402214,-481.000950,579.431344,-481.000950,579.431344,425.010135,363.526323,NaN,46075.142857,3.491166e+09,0.847945,75705.155679,NaN,11.211926,54337.42660,1,1086.724850,1075.581822,3738.815886,2652.091036,0.709340,3.065097e+07
2,1003,1.0,3.0,1.206121e+07,6.030604e+06,16735.714286,-102.704911,27.019851,12.536194,19.778022,-410.380266,526.725525,-410.380266,526.725525,621.936260,544.350886,NaN,55283.142857,4.358511e+09,0.598719,78733.145871,NaN,11.211926,92335.78323,1,1092.229665,1081.030192,3757.754893,2665.525228,0.709340,4.460947e+07
3,1004,1.0,4.0,2.692258e+06,1.346129e+06,3788.142857,-102.297038,26.587408,12.486214,19.536806,-490.240391,594.640076,-490.240391,594.640076,444.545795,375.941963,NaN,15363.000000,1.449504e+09,1.196649,94221.940180,NaN,11.211926,12838.35445,1,1077.104114,1066.059735,3705.716284,2628.612170,0.709340,9.957558e+06
4,1005,1.0,5.0,2.317725e+07,1.158863e+07,30596.571429,-102.445700,26.906952,12.882205,19.894577,-417.765529,539.088928,-417.765529,539.088928,534.694978,465.168451,NaN,112079.000000,1.318374e+10,2.242050,117305.832929,NaN,11.211926,49989.50916,1,1148.038759,1136.267033,3949.762951,2801.724192,0.709340,8.572315e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,32.0,54.0,2.460871e+06,1.230436e+06,3922.714286,-101.714324,26.174618,12.658807,19.416714,-514.517334,580.405287,-514.517334,580.405287,357.454945,293.721267,NaN,18889.000000,1.104084e+09,0.508554,58432.484499,NaN,11.268627,37142.59866,1,950.757149,941.008303,3221.130693,2270.373544,0.704837,8.906027e+06
2465,32055,32.0,55.0,5.502683e+06,2.751341e+06,8847.000000,-102.851571,26.542453,12.525792,19.534122,-447.884024,545.919163,-447.884024,545.919163,535.403613,468.461905,NaN,29902.000000,2.396876e+09,0.138660,80134.667680,NaN,11.268627,215650.56020,1,942.639988,932.974374,3193.630047,2250.990059,0.704837,1.991451e+07
2466,32056,32.0,56.0,2.880959e+07,1.440480e+07,37702.714286,-102.678055,25.746405,11.916062,18.831230,-505.051806,568.709394,-505.051806,568.709394,433.556164,378.290362,NaN,142958.571429,2.484715e+10,3.272704,173750.774043,NaN,11.268627,43682.10105,1,1158.062290,1146.187785,3923.472982,2765.410692,0.704837,1.042635e+08
2467,32057,32.0,57.0,1.371637e+07,6.858186e+06,15674.571429,-102.312009,26.272720,12.047195,19.159957,-538.027021,602.355024,-538.027021,602.355024,406.955520,351.221659,NaN,18421.428571,1.414295e+09,0.843291,76845.878313,NaN,11.268627,21844.70067,1,1326.206067,1312.607453,4493.137990,3166.931923,0.704837,4.964030e+07
